# Importe de librerias

In [29]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from nltk.corpus import stopwords
import collections
from sklearn.preprocessing import StandardScaler

#descargas
#nltk.download('stopwords')
#nltk.download('punkt')

# Ampliacion del dataset

In [30]:
ds_descrip = pd.read_csv('properati_argentina_2021_decrip.csv')

In [31]:
ds_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


In [32]:
ds_descrip.dtypes

id                      object
property_description    object
dtype: object

In [33]:
ds_descrip.shape

(460154, 2)

Realizo transformaciones sobre el dataset


In [34]:
descriptions = ds_descrip['property_description'].astype(str)
descriptions = descriptions.str.replace('[{}]'.format(string.punctuation), '')

C:\Users\EQUIPO~1\AppData\Local\Temp/ipykernel_18964/642431205.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  descriptions = descriptions.str.replace('[{}]'.format(string.punctuation), '')


In [35]:
descriptions

0         Corredor Responsable MARIA ALEJANDRA GENOVEVA ...
1         ¡HERMOSO PH TIPO CASA DE 2 AMBIENTESbrAPTO CRÉ...
2         Excelente PH en Núñez 4 amb Oportunidad  Total...
3         UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO ...
4         Ph 2 Amb Villa Crespo exclusivo uso profesiona...
                                ...                        
460149    Corredor Responsable Hector Garaygorta  Cristi...
460150    Corredor Responsable MARIA DEL ROSARIO NEME  1...
460151    En cumplimiento de las leyes provinciales vige...
460152    En cumplimiento de las leyes provinciales vige...
460153    CASA EN VENTA EN BARILOCHE – Lomas del Cauquén...
Name: property_description, Length: 460154, dtype: object

### Combino dataset de descripciones con dataset popiedades obtenidos TP1

A traves del ID de cada descripcion puedo saber de que propiedad es la descipcion de cada fila

In [36]:
ds_train= pd.read_csv('properati_argentina_train.csv')
ds_test = pd.read_csv('properati_argentina_test.csv')

In [37]:
ds_train.head()

,Unnamed: 0,id,latitud,longitud,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,precioxm2_relativo_q
0,0,CLj2GNCsBreAzTKFXCaLIQ==,-34.601807,-58.469965,Departamento,2.0,1.0,53.0,49.0,80000.0,bajo
1,1,TvqKIPzzT1urYfVhh8tC7w==,-34.580088,-58.424965,Departamento,2.0,1.0,35.0,35.0,79900.0,medio
2,2,QRs3RUjw1RS5/WRne8ApbA==,-34.551481,-58.452358,Departamento,1.0,1.0,25.0,20.0,69000.0,medio
3,3,ju90QBsV36i7DCOOWCJpaA==,-34.632290,-58.464844,Departamento,5.0,3.0,75.0,75.0,150000.0,medio
4,4,uxD4SK3jtGbmxfIlN03UMQ==,-34.549279,-58.463074,Departamento,2.0,1.0,46.0,34.0,85000.0,bajo


In [25]:
#Eliminamos columna Unnamed

ds_train.drop(columns="Unnamed: 0", inplace=True)
ds_test.drop(columns="Unnamed: 0", inplace=True)

In [38]:
#Dataset train
ds_train = ds_train.join(ds_descrip.set_index('id'), on='id')
ds_train.head()

#Dataset test
ds_test = ds_test.join(ds_descrip.set_index('id'), on='id')

## Seleccion de atributos de propiedades

- Seleccionamos atributos que podrian llegar a aportar informacion importante al dataset
- A traves de regex buscamos palabras claves en las descripciones de las propiedades
- Colocamos un "1" si la propiedad posee esa palabra, de lo contrario "0"

### Pileta

In [42]:
regexp = re.compile(r'piscina|pileta/gi', re.IGNORECASE)

ds_train['piscina'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['piscina'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen pileta

In [43]:
ds_train["piscina"].value_counts()

0    60313
1     7699
Name: piscina, dtype: int64

### Balcon 

In [44]:
regexp = re.compile(r'balcon', re.IGNORECASE)

ds_train['balcon'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['balcon'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen balcon

In [46]:
ds_train["balcon"].value_counts()

0    55956
1    12056
Name: balcon, dtype: int64

### Patio

In [52]:
regexp = re.compile(r'patio', re.IGNORECASE)

ds_train['patio'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['patio'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen patio

In [53]:
ds_train["patio"].value_counts()

0    57091
1    10921
Name: patio, dtype: int64

### Terraza

In [54]:
regexp = re.compile(r'terraza', re.IGNORECASE)

ds_train['terraza'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['terraza'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen terraza

In [55]:
ds_train["terraza"].value_counts()

0    47376
1    20636
Name: terraza, dtype: int64

### Luminoso

In [60]:
regexp = re.compile(r'luminosa|luminoso/gi', re.IGNORECASE)

ds_train['luminoso'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['luminoso'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades son caracterizadas como luminosas

In [61]:
ds_train["luminoso"].value_counts()

0    65280
1     2732
Name: luminoso, dtype: int64

### Gimnasio

In [64]:
regexp = re.compile(r'gimnasio', re.IGNORECASE)

ds_train['gimnasio'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['gimnasio'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

Verifico cuantas propiedades tienen gimnasio

In [65]:
ds_train["gimnasio"].value_counts()

0    62685
1     5327
Name: gimnasio, dtype: int64

### Garaje 

In [66]:
regexp = re.compile(r'garage|cochera/gi', re.IGNORECASE)

ds_train['garaje'] = ds_train['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)
ds_test['garaje'] = ds_test['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [67]:
ds_train["garaje"].value_counts()

0    67398
1      614
Name: garaje, dtype: int64

# Modelos - XGboost